In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
path = "/content/drive/MyDrive"
os.chdir(path)

In [3]:
features = pd.read_csv('mfcc_mean_caca.csv')

In [4]:
label = features.loc[:, 'emotion'].values

In [372]:
index = ['emotion'] + \
          ['mfcc_' + str(i+1) + '_mean' for i in range(50)] + \
          ['mfcc_' + str(i+1) + '_std' for i in range(50)]
          #['contrast' + str(i+1) + '_mean' for i in range(7)]
          #['chroma' + str(i+1) + '_mean' for i in range(12)] + \
          #['mels' + str(i+1) + '_mean' for i in range(128)] + \
mfcc_ind = index[1:]
mfccs = features.loc[:, mfcc_ind].values

In [373]:
mfccs.shape
label.shape

(1125,)

In [374]:
# Make Number Label (1125)
Y = []
for emo in range(0, len(label)):
    if label[emo] == 'angry':
        Y.append(0)
    if label[emo] == 'calm':
        Y.append(1)
    if label[emo] == 'disgust':
        Y.append(2)
    if label[emo] == 'fearful':
        Y.append(3)
    if label[emo] == 'happy':
        Y.append(4)
    if label[emo] == 'neutral':
        Y.append(5)
    if label[emo] == 'sad':
        Y.append(6)
    if label[emo] == 'surprised':
        Y.append(7)
Y = np.array(Y)

In [375]:
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, BatchNormalization
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.utils import plot_model

In [532]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(mfccs, Y, test_size=0.1)

In [535]:
x_traincnn = np.expand_dims(X_train, axis=2)
x_testcnn = np.expand_dims(X_test, axis=2)

In [536]:
x_traincnn.shape

(1012, 100, 1)

In [ ]:
model = Sequential()
model.add(Conv1D(32, 2,
                 input_shape=x_traincnn.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling1D())

model.add(Conv1D(64, 2,))
model.add(Activation('relu'))
model.add(MaxPooling1D())

model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation = 'softmax'))
#opt = tf.keras.optimizers.RMSprop(lr=0.0005, rho=0.9, epsilon=1e-07, decay=0.0)
opt=tf.keras.optimizers.Adam()
model.summary()


In [428]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [537]:
history=model.fit(x_traincnn, y_train, epochs=30, validation_data=(x_testcnn, y_test))

Epoch 1/30
32/32 [==============================] - 0s 12ms/step - loss: 0.6153 - accuracy: 0.7836 - val_loss: 0.1701 - val_accuracy: 0.9735
Epoch 2/30
32/32 [==============================] - 1s 16ms/step - loss: 0.6095 - accuracy: 0.7826 - val_loss: 0.1556 - val_accuracy: 0.9735
Epoch 3/30
32/32 [==============================] - 0s 15ms/step - loss: 0.6413 - accuracy: 0.7609 - val_loss: 0.1646 - val_accuracy: 0.9912
Epoch 4/30
32/32 [==============================] - 0s 10ms/step - loss: 0.6124 - accuracy: 0.7757 - val_loss: 0.1534 - val_accuracy: 0.9823
Epoch 5/30
32/32 [==============================] - 0s 11ms/step - loss: 0.6887 - accuracy: 0.7381 - val_loss: 0.1768 - val_accuracy: 0.9823
Epoch 6/30
32/32 [==============================] - 0s 12ms/step - loss: 0.6261 - accuracy: 0.7609 - val_loss: 0.1690 - val_accuracy: 0.9646
Epoch 7/30
32/32 [==============================] - 0s 11ms/step - loss: 0.5929 - accuracy: 0.7787 - val_loss: 0.1815 - val_accuracy: 0.9823
Epoch 8/30
32

In [122]:
path = "/content/drive/MyDrive/ELEC301/test"
os.chdir(path)

In [34]:
features_test = pd.read_csv('testcaca.csv')

In [35]:
NUM = features_test.loc[:, 'emotion'].values

In [ ]:
NUM

In [ ]:
index = ['emotion'] + \
          ['mfcc_' + str(i+1) + '_mean' for i in range(50)] + \
          ['mfcc_' + str(i+1) + '_std' for i in range(50)] 
          #['contrast' + str(i+1) + '_mean' for i in range(7)]
          #['chroma' + str(i+1) + '_mean' for i in range(12)] + \
          #['mels' + str(i+1) + '_mean' for i in range(128)] + \
test_ind = index[1:]
test = features_test.loc[:, test_ind].values

In [538]:
predictions = model.predict(test)
score = tf.nn.softmax(predictions)

10/10 [==============================] - 0s 3ms/step


In [539]:
label_predict = []
for i in range(315):
  label_predict.append(np.argmax(score[i]))
res = []

In [ ]:
score

In [541]:
for emo in range(0, len(label_predict)):
    if label_predict[emo] == 0:
      res.append('angry')
    if label_predict[emo] == 1:
      res.append('calm')
    if label_predict[emo] == 2:
      res.append('disgust')
    if label_predict[emo] == 3:
      res.append('fearful')
    if label_predict[emo] == 4:
      res.append('happy')
    if label_predict[emo] == 5:
      res.append('neutral')
    if label_predict[emo] == 6:
      res.append('sad')
    if label_predict[emo] == 7:
      res.append('surprised')
print(res)
res = np.array(res).transpose()
NUM = np.array(NUM).transpose()

['surprised', 'angry', 'happy', 'angry', 'calm', 'happy', 'calm', 'neutral', 'surprised', 'angry', 'angry', 'angry', 'calm', 'disgust', 'angry', 'sad', 'sad', 'calm', 'calm', 'happy', 'fearful', 'surprised', 'fearful', 'surprised', 'angry', 'happy', 'surprised', 'sad', 'calm', 'fearful', 'sad', 'fearful', 'happy', 'neutral', 'angry', 'surprised', 'sad', 'fearful', 'angry', 'angry', 'calm', 'calm', 'disgust', 'angry', 'calm', 'surprised', 'fearful', 'fearful', 'angry', 'sad', 'angry', 'fearful', 'surprised', 'surprised', 'disgust', 'sad', 'happy', 'calm', 'happy', 'fearful', 'calm', 'calm', 'disgust', 'happy', 'angry', 'fearful', 'angry', 'neutral', 'fearful', 'surprised', 'calm', 'happy', 'angry', 'angry', 'happy', 'angry', 'calm', 'sad', 'calm', 'calm', 'fearful', 'angry', 'happy', 'surprised', 'disgust', 'sad', 'happy', 'calm', 'neutral', 'disgust', 'disgust', 'happy', 'angry', 'calm', 'angry', 'sad', 'angry', 'disgust', 'happy', 'happy', 'sad', 'calm', 'happy', 'disgust', 'sad', 'an

In [ ]:
sample = np.column_stack([NUM, res])
sort_sample = sample[sample[:, 0].argsort()]
name = []
for item in sort_sample[:,0]:
  item = str(item)
  name.append(item)
name = np.array(name).transpose()
sort_sample[:,0] = name
sort_sample

In [543]:
res_chroma = sort_sample

In [544]:
import pandas as pd
DF = pd.DataFrame(sort_sample)
DF.to_csv("res1213_all.csv")